In [ ]:
from Core import Workspace
import Logger

logger = Logger.console_logger(0, 'INFO')
context = Workspace(r'c:\data\sweep\ws', logger)

In [ ]:
def generate_something(value1, value2):
    return {'key1': value1, "key2": value2}

In [ ]:
context.run(generate_something, 1, 2)
#pseudo-code
#   path = md5("generate_somehting" + "1" + "2")
#   if not path.exists:
#       result = generate_something(1, 2)
#       json.dump(result, path)
#       return result
#   else:
#       return json.read(path)

In [ ]:
vw_path = r'C:\src\VowpalWabbit\vowpal_wabbit\vowpalwabbit\out\target\Release\x64\vw.exe'
from Vw import Vw, VwInput

vw = Vw(vw_path, context, procs = 1)

In [ ]:
opts = {'#cmd': '--ccb_explore_adf --epsilon 0.1 --dsjson', '-d': r'C:\data\sweep\1.json'}
vw.run(opts)

In [ ]:
inputs = [r'C:\data\sweep\1.json', r'C:\data\sweep\2.json', r'C:\data\sweep\3.json']

opts_in = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson'}
opts_out = ['--cache_file']

result = vw.test(inputs, opts_in, opts_out)
#pseudocode:
#    for i in inputs:
#       opts_in['-d'] = i
#       populated = populate(opts_in, opts_out)
#       yield return (vw.run(opts_in, populated), populated)

logger.info('Done')
print(result.Loss)
print(result.Populated)

In [ ]:
inputs = [r'C:\data\sweep\1.json', r'C:\data\sweep\3.json']

opts_in = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume'}
opts_out = ['-f']

result = vw.train(inputs, opts_in, opts_out)
#pseudocode:
#    for i in inputs:
#       opts_in['-d'] = i
#       populated = populate(opts_in, opts_out)
#       yield return (vw.run(opts_in, populated), populated)
#       opts_in['-i'] = populated['-f']

logger.info('Done')
print(result.Loss)
print(result.Populated)

In [ ]:
opts_in_1 = {'#base': '--ccb_explore_adf --epsilon 0.1 --dsjson --save_resume'}
opts_in_2 = {'#base': '--ccb_explore_adf --epsilon 0.2 --dsjson --save_resume'}
opts_out = ['-f']

results_par = vw.train(inputs, [opts_in_1, opts_in_2], opts_out)
results_par

In [ ]:
from VwSweep import VwSweep
import VwOptsGrid

sweep = VwSweep(vw)
inputs = [r'C:\data\sweep\1.json', r'C:\data\sweep\2.json']

In [ ]:
opts1 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume', '-l': 0.1}
opts2 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume', '-l': 0.01}
opts3 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume', '-l': 0.001}

opts = [opts1, opts2]
results = sweep.iteration(inputs, opts)

print('Best loss: {0}'.format(results[0].Loss))
print('Best options: {0}'.format(results[0].Opts))

In [ ]:
from VwOptsGrid import dimension, product, Grid, Configuration
def generate_grid():
    hyper_points_1 = product(
        dimension('-l', [1e-6, 1e-5]),
        dimension('--cb_type', ['ips', 'mtr']),
    )
    hyper_points_2 = dimension('--l1', [1e-09, 1e-07, 1e-5])

    return [
        Grid(hyper_points_1, Configuration(name='Step1', output=1, promote=1)),
        Grid(hyper_points_2, Configuration(name='Step2', output=1, promote=1))
    ]

In [ ]:
grid = generate_grid()
base_command = {'#base': '--ccb_explore_adf --epsilon 0.2 --dsjson --save_resume --preserve_performance_counters'}

result = sweep.run(inputs, grid, base_command)

In [ ]:
result[0].Opts